In [1]:
import numpy as np
import matplotlib.pyplot as plt
import qutip as qtp
from IPython.display import display, Math, Latex

In [2]:
a = qtp.qutrit_basis()


In [3]:
print(a[0]*qtp.dag(a[0]))

Quantum object: dims = [[3], [3]], shape = (3, 3), type = oper, isherm = True
Qobj data =
[[ 1.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


## Hamiltonian of two qutrits coupled directly

$$ H^{direct} = \Sum_{j = A,B} [\omega_{j}\ket{1}\bra{1} +(2\omega_{j} - \Delta_{j})\ket{2}\bra{2}] + g[\ket{01}\bra{10} + \sqrt(2) \ket{02}{11} + 2\ket{12}\bra{21} + H.C.] $$


In [17]:
def basis_kets():
    ket0 = qtp.basis(3, 0)
    ket1 = qtp.basis(3, 1)
    ket2 = qtp.basis(3, 2)
    bra0 = qtp.dag(ket0)
    bra1 = qtp.dag(ket1)
    bra2 = qtp.dag(ket2)
    return ket0,ket1,ket2,bra0,bra1,bra2
    

In [30]:
def basisQubit():
    ket0 = qtp.basis(2, 0)
    ket1 = qtp.basis(2, 1)
    bra0 = qtp.dag(ket0)
    bra1 = qtp.dag(ket1)
    return ket0,ket1,bra0,bra1

In [37]:
def proyector():
    bitKet0, bitKet1, bitBra0, bitBra1 = basisQubit()
    triKet0, triKet1, triKet2, triBra0, triBra1, triBra2 = basis_kets()
    P = bitKet0*triBra0 + bitKet1*triBra1
    return P

In [18]:
def qubit_term(freq1, anh1, freq2, anh2):
    ket0,ket1,ket2,bra0,bra1,bra2 = basis_kets()
    qubit_term1 = freq1*ket1*bra1 + (2*freq1 - anh1)*ket2*bra2
    qubit_term2 = freq2*ket1*bra1 + (2*freq2 - anh2)*ket2*bra2
    qubit_term = qtp.tensor(qubit_term1,qubit_term2)
    return qubit_term
    
    

In [19]:
def coupling_hamiltonian(coupling):
    ket0,ket1,ket2,bra0,bra1,bra2 = basis_kets()
    term1 = (qtp.tensor(ket0,ket1)*qtp.tensor(bra1,bra0) + np.sqrt(2)*qtp.tensor(ket0,ket2)*qtp.tensor(bra1,bra1)
             + np.sqrt(2)*qtp.tensor(ket2,ket0)*qtp.tensor(bra1,bra1) + 2* qtp.tensor(ket1,ket2)*qtp.tensor(bra2,bra1))
    term2 = qtp.dag(term1)
    
    return coupling*(term1 + term2)
    
    
    

In [20]:
def dir_coupled_qutrit(freq1, anh1, freq2, anh2, coupling):
    """
    freq 1 , 2 : splitting between bottom two levels (in GHZ)
    anh 1,2 : anharmonicity of two qubits in MHz
    g : coupling between qutrits 
    """
    qubit_term_hamiltonian = qubit_term(freq1, anh1, freq2, anh2)
    coupling_term = coupling_hamiltonian(coupling)
    hamiltonian_direct = qubit_term_hamiltonian + coupling_term
    return hamiltonian_direct
    
    
    
    
    

In [21]:
coupling = 0.3
dir_coupled_qutrit(5, .5, 5, .5, .3)

Quantum object: dims = [[3, 3], [3, 3]], shape = (9, 9), type = oper, isherm = True
Qobj data =
[[  0.           0.           0.           0.           0.           0.
    0.           0.           0.        ]
 [  0.           0.           0.           0.3          0.           0.
    0.           0.           0.        ]
 [  0.           0.           0.           0.           0.42426407   0.
    0.           0.           0.        ]
 [  0.           0.3          0.           0.           0.           0.
    0.           0.           0.        ]
 [  0.           0.           0.42426407   0.          25.           0.
    0.42426407   0.           0.        ]
 [  0.           0.           0.           0.           0.          47.5
    0.           0.6          0.        ]
 [  0.           0.           0.           0.           0.42426407   0.
    0.           0.           0.        ]
 [  0.           0.           0.           0.           0.           0.6
    0.          47.5          0.

In [48]:
evolution_time = np.pi/(coupling)

In [49]:
U_evolution = (-1j * dir_coupled_qutrit(5, .5, 5, .5, .3) * evolution_time).expm()

In [50]:
U_evolution

Quantum object: dims = [[3, 3], [3, 3]], shape = (9, 9), type = oper, isherm = False
Qobj data =
[[ 1.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j        ]
 [ 0.00000000+0.j         -1.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000+0.j          0.99394340+0.07529449j
   0.00000000+0.j         -0.02293430+0.01324112j  0.00000000+0.j
  -0.00605660+0.07529449j  0.00000000+0.j          0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000+0.j          0.00000000+0.j
  -1.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000+0.j         -0.02293430+0.013

In [51]:
# Not works
def qutrit_to_qubit(U_evolution):
    ket0,ket1,ket2,bra0,bra1,bra2 = basis_kets()
    P = qtp.tensor(ket0,ket0)*qtp.dag(qtp.tensor(ket0,ket0)) + qtp.tensor(ket0,ket1)*qtp.dag(qtp.tensor(ket0,ket1)) + qtp.tensor(ket1,ket0)*qtp.dag(qtp.tensor(ket1,ket0)) + qtp.tensor(ket1,ket1)*qtp.dag(qtp.tensor(ket1,ket1))
    return qtp.dag(P)*(U_evolution)*(P)
    
    

In [52]:
U_qubit = qutrit_to_qubit(U_evolution)

In [53]:
U_qubit

Quantum object: dims = [[3, 3], [3, 3]], shape = (9, 9), type = oper, isherm = False
Qobj data =
[[ 1.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j        ]
 [ 0.00000000+0.j         -1.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000+0.j          0.00000000+0.j
  -1.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.00000000+0.j          0.00000000+0.j          0.00000000+0.j        ]
 [ 0.00000000+0.j          0.00000000+0.j          0.00000000+0.j
   0.0000

In [54]:
P = proyector()
P = qtp.tensor(P, P)
P

Quantum object: dims = [[2, 2], [3, 3]], shape = (4, 9), type = other
Qobj data =
[[ 1.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  1.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  1.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  1.  0.  0.  0.  0.]]

In [55]:
 iS = P * U_evolution * qtp.dag(P)

In [60]:
iS * qtp.dag(iS)

Quantum object: dims = [[2, 2], [2, 2]], shape = (4, 4), type = oper, isherm = True
Qobj data =
[[ 1.          0.          0.          0.        ]
 [ 0.          1.          0.          0.        ]
 [ 0.          0.          1.          0.        ]
 [ 0.          0.          0.          0.99859738]]